<a href="https://colab.research.google.com/github/lonerpatti/BigDataPos/blob/main/Uni_Facef_DS_Regress%C3%A3o_Linear_com_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'>Uni-Facef - DS - Regressão Linear com Pyspark </font>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Salary_Data.csv to Salary_Data.csv


In [3]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 37.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=d8080ece58b869e97791302992cf11279bd106276aa019a50306c0e9f01eff6e
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


Treinamento de um modelo com Regressão Linear usando PySpark

   O objetivo da Regressão Linear é prever os valores de uma variável dependente com base em resultados da variável independente. Então, da fórmula de regressão linear Y = ax + b, "x" é a variável independente (preditora) e y é a variável dependente (resposta), uma vez que "y" depende de "x". É um algoritmo de Aprendizado Supervisionado.

Baseado no artigo: https://www.cetax.com.br/blog/tutorial-pyspark-e-mllib/

In [4]:
# Import SparkSession
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .getOrCreate()
   
sc = spark.sparkContext

In [5]:
df = spark.read \
    .option("delimiter", ",") \
    .csv('Salary_Data.csv', header=False)

df.show()

+---+--------+
|_c0|     _c1|
+---+--------+
|1.1|39343.00|
|1.3|46205.00|
|1.5|37731.00|
|2.0|43525.00|
|2.2|39891.00|
|2.9|56642.00|
|3.0|60150.00|
|3.2|54445.00|
|3.2|64445.00|
|3.7|57189.00|
|3.9|63218.00|
|4.0|55794.00|
|4.0|56957.00|
|4.1|57081.00|
|4.5|61111.00|
|4.9|67938.00|
|5.1|66029.00|
|5.3|83088.00|
|5.9|81363.00|
|6.0|93940.00|
+---+--------+
only showing top 20 rows



#### Renomeia os campos do dataframe

In [6]:
df = df \
    .withColumnRenamed("_c0", "YearsExperience") \
    .withColumnRenamed("_c1", "Salary") \
    .select("Salary", "YearsExperience")

df.printSchema()

root
 |-- Salary: string (nullable = true)
 |-- YearsExperience: string (nullable = true)



In [7]:
from pyspark.sql.types import *


# Função para modifica o tipo de uma lista de colunas
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 


# Lista das colunas para coversão
columns = ['YearsExperience', 'Salary']


# Converte as colunas para "FloatType()"
df = convertColumn(df, columns, FloatType())

df.printSchema()

root
 |-- Salary: float (nullable = true)
 |-- YearsExperience: float (nullable = true)



In [8]:
df.show()

+-------+---------------+
| Salary|YearsExperience|
+-------+---------------+
|39343.0|            1.1|
|46205.0|            1.3|
|37731.0|            1.5|
|43525.0|            2.0|
|39891.0|            2.2|
|56642.0|            2.9|
|60150.0|            3.0|
|54445.0|            3.2|
|64445.0|            3.2|
|57189.0|            3.7|
|63218.0|            3.9|
|55794.0|            4.0|
|56957.0|            4.0|
|57081.0|            4.1|
|61111.0|            4.5|
|67938.0|            4.9|
|66029.0|            5.1|
|83088.0|            5.3|
|81363.0|            5.9|
|93940.0|            6.0|
+-------+---------------+
only showing top 20 rows



In [9]:
df.groupBy("Salary").count().sort("Salary",ascending=False).show()

+--------+-----+
|  Salary|count|
+--------+-----+
|122391.0|    1|
|121872.0|    1|
|116969.0|    1|
|113812.0|    1|
|112635.0|    1|
|109431.0|    1|
|105582.0|    1|
|101302.0|    1|
| 98273.0|    1|
| 93940.0|    1|
| 91738.0|    1|
| 83088.0|    1|
| 81363.0|    1|
| 67938.0|    1|
| 66029.0|    1|
| 64445.0|    1|
| 63218.0|    1|
| 61111.0|    1|
| 60150.0|    1|
| 57189.0|    1|
+--------+-----+
only showing top 20 rows



#### O método "describe()"

Faz a descrição do "df" baseado nas colunas, retornando uma contagem dos elementos, a média, desvio padrão, valores mínimo e máximo.

In [10]:
df.describe().show()

+-------+------------------+------------------+
|summary|            Salary|   YearsExperience|
+-------+------------------+------------------+
|  count|                30|                30|
|   mean|           76003.0|5.3133333643277485|
| stddev|27414.429784582302|2.8378881722287805|
|    min|           37731.0|               1.1|
|    max|          122391.0|              10.5|
+-------+------------------+------------------+



#### Módulo DenseVector

O uso do DenseVector é uma maneira otimizada de lidar com valores numéricos, acelerando o processamento realizado pelo Spark.

In [11]:
from pyspark.ml.linalg import DenseVector

# Define o RDD "input_data" para aplicar DenseVector 
input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
df = spark.createDataFrame(input_data, ["label", "features"])

df.show()

+-------+--------------------+
|  label|            features|
+-------+--------------------+
|39343.0| [1.100000023841858]|
|46205.0|[1.2999999523162842]|
|37731.0|               [1.5]|
|43525.0|               [2.0]|
|39891.0| [2.200000047683716]|
|56642.0|[2.9000000953674316]|
|60150.0|               [3.0]|
|54445.0| [3.200000047683716]|
|64445.0| [3.200000047683716]|
|57189.0| [3.700000047683716]|
|63218.0|[3.9000000953674316]|
|55794.0|               [4.0]|
|56957.0|               [4.0]|
|57081.0| [4.099999904632568]|
|61111.0|               [4.5]|
|67938.0| [4.900000095367432]|
|66029.0| [5.099999904632568]|
|83088.0| [5.300000190734863]|
|81363.0| [5.900000095367432]|
|93940.0|               [6.0]|
+-------+--------------------+
only showing top 20 rows



####  Colocando os dados na mesma escala com método StantardScaler

O algoritmo de Regressão Linear trabalha com distâncias euclidianas, ele realiza operações de distância entre pontos no plano cartesiano. Isso exige que os dados estejam na mesma escala.

Com o StantardScaler, subtrai-se a média do valor "x" definido, e divide-se pela diferença (xmax – xmin). Desta forma, os dados estarão distribuidos ao longo de 0 na mesma escala.

In [12]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler


# Cria a instância do objeto
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit no DataFrame
scaler = standardScaler.fit(df)

# Transforma em dataframe com a escala
scaled_df = scaler.transform(df)

print(type(scaled_df))

# Inspect the result
scaled_df.take(2)

<class 'pyspark.sql.dataframe.DataFrame'>


[Row(label=39343.0, features=DenseVector([1.1]), features_scaled=DenseVector([0.3876])),
 Row(label=46205.0, features=DenseVector([1.3]), features_scaled=DenseVector([0.4581]))]

In [13]:
help(scaled_df.take)

Help on method take in module pyspark.sql.dataframe:

take(num) method of pyspark.sql.dataframe.DataFrame instance
    Returns the first ``num`` rows as a :class:`list` of :class:`Row`.
    
    >>> df.take(2)
    [Row(age=2, name='Alice'), Row(age=5, name='Bob')]
    
    .. versionadded:: 1.3



In [14]:
scaled_df.show()

+-------+--------------------+--------------------+
|  label|            features|     features_scaled|
+-------+--------------------+--------------------+
|39343.0| [1.100000023841858]|[0.3876121809894839]|
|46205.0|[1.2999999523162842]| [0.458087096256266]|
|37731.0|               [1.5]|[0.5285620535293859]|
|43525.0|               [2.0]|[0.7047494047058479]|
|39891.0| [2.200000047683716]|[0.7752243619789678]|
|56642.0|[2.9000000953674316]|[1.0218866704285499]|
|60150.0|               [3.0]|[1.0571241070587718]|
|54445.0| [3.200000047683716]|[1.1275990643318918]|
|64445.0| [3.200000047683716]|[1.1275990643318918]|
|57189.0| [3.700000047683716]|[1.3037864155083538]|
|63218.0|[3.9000000953674316]|[1.3742613727814736]|
|55794.0|               [4.0]|[1.4094988094116958]|
|56957.0|               [4.0]|[1.4094988094116958]|
|57081.0| [4.099999904632568]| [1.444736246041918]|
|61111.0|               [4.5]|[1.5856861605881578]|
|67938.0| [4.900000095367432]|[1.7266360751343977]|
|66029.0| [5

#### Slit em dados de treino e teste

In [15]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.75,.25],seed=1234)

train_data.show()

+--------+--------------------+--------------------+
|   label|            features|     features_scaled|
+--------+--------------------+--------------------+
| 37731.0|               [1.5]|[0.5285620535293859]|
| 39891.0| [2.200000047683716]|[0.7752243619789678]|
| 43525.0|               [2.0]|[0.7047494047058479]|
| 54445.0| [3.200000047683716]|[1.1275990643318918]|
| 55794.0|               [4.0]|[1.4094988094116958]|
| 56642.0|[2.9000000953674316]|[1.0218866704285499]|
| 56957.0|               [4.0]|[1.4094988094116958]|
| 57081.0| [4.099999904632568]| [1.444736246041918]|
| 57189.0| [3.700000047683716]|[1.3037864155083538]|
| 60150.0|               [3.0]|[1.0571241070587718]|
| 61111.0|               [4.5]|[1.5856861605881578]|
| 63218.0|[3.9000000953674316]|[1.3742613727814736]|
| 64445.0| [3.200000047683716]|[1.1275990643318918]|
| 66029.0| [5.099999904632568]|[1.7971109483948418]|
| 67938.0| [4.900000095367432]|[1.7266360751343977]|
| 81363.0| [5.900000095367432]|[2.079010777487

#### Instanciando o algoritmo e treinando o modelo (fit)

In [16]:
# Import `LinearRegression`
from pyspark.ml.regression import LinearRegression

# Initialize `lr`
lr = LinearRegression(labelCol="label", maxIter=10)

# Fit the data to the model
linearModel = lr.fit(train_data)

type(linearModel)

pyspark.ml.regression.LinearRegressionModel

#### Fazendo a predição com os dados de Treino

Criará a nova coluna "prediction" que dá para confrontar com a coluna "label" (variável resposta de treino)

In [17]:
# Generate predictions
predicted = linearModel.transform(test_data)

predicted.show()

+--------+--------------------+--------------------+------------------+
|   label|            features|     features_scaled|        prediction|
+--------+--------------------+--------------------+------------------+
| 39343.0| [1.100000023841858]|[0.3876121809894839]| 35027.48409440431|
| 46205.0|[1.2999999523162842]| [0.458087096256266]|36917.726105474954|
| 98273.0| [7.099999904632568]|[2.5018603531006898]| 91734.76357995521|
|109431.0| [8.699999809265137]|[3.0656598432602977]|106856.70417521015|
|112635.0| [9.600000381469727]|[3.3827972770083514]|115362.80167507136|
|113812.0| [8.199999809265137]| [2.889472492083836]|102131.09745752487|
|116969.0|               [9.5]|[3.3475596723527774]|114417.67672618249|
|122391.0|[10.300000190734863]|[3.6294595014452575]|121978.64927715484|
+--------+--------------------+--------------------+------------------+



#### Coeficientes da equação

Visto que a equação é "Y = ax + b"

Onde "coefficients" é o valor de "a", e "intercept" é o valor de "b".

In [18]:
# Coefficients para o modelo
linearModel.coefficients

DenseVector([9451.2134])

In [19]:
# Intercept para o modelo
linearModel.intercept

24631.149090162206

#### Erro Quadrático Médio, em inglês MSE, neste caso sendo representado pela raiz quadrada deste valor(RMSE)

In [20]:
# Get the RMSE
linearModel.summary.rootMeanSquaredError

5532.104709071577

####  Métrica de avaliação do modelo

Extrai-se o R2, ou Coeficiente de Determinação, uma métrica estatística de proximidade de pontos e reta sobreposta

In [21]:
linearModel.summary.r2

0.9347037587962345

### Documentação

https://spark.apache.org/docs/latest/api/python/index.html

https://spark.apache.org/docs/latest/ml-classification-regression.html